# WannaCry Ransomware
Detects WannaCry ransomware activity

## Rule Content
```
- title: WannaCry Ransomware
  id: 41d40bff-377a-43e2-8e1b-2e543069e079
  status: experimental
  description: Detects WannaCry ransomware activity
  references:
  - https://www.hybrid-analysis.com/sample/ed01ebfbc9eb5bbea545af4d01bf5f1071661840480439c6e5babe8e080e41aa?environmentId=100
  author: Florian Roth (rule), Tom U. @c_APT_ure (collection)
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection1:
      Image:
      - '*\tasksche.exe'
      - '*\mssecsvc.exe'
      - '*\taskdl.exe'
      - '*\@WanaDecryptor@*'
      - '*\WanaDecryptor*'
      - '*\taskhsvc.exe'
      - '*\taskse.exe'
      - '*\111.exe'
      - '*\lhdfrgui.exe'
      - '*\diskpart.exe'
      - '*\linuxnew.exe'
      - '*\wannacry.exe'
    selection2:
      CommandLine:
      - '*icacls * /grant Everyone:F /T /C /Q*'
      - '*bcdedit /set {default} recoveryenabled no*'
      - '*wbadmin delete catalog -quiet*'
      - '*@Please_Read_Me@.txt*'
    condition: 1 of them
  fields:
  - CommandLine
  - ParentCommandLine
  falsepositives:
  - Diskpart.exe usage to manage partitions on the local hard drive
  level: critical

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='(process_path.keyword:(*\\tasksche.exe OR *\\mssecsvc.exe OR *\\taskdl.exe OR *\\@WanaDecryptor@* OR *\\WanaDecryptor* OR *\\taskhsvc.exe OR *\\taskse.exe OR *\\111.exe OR *\\lhdfrgui.exe OR *\\diskpart.exe OR *\\linuxnew.exe OR *\\wannacry.exe) OR process_command_line.keyword:(*icacls\ *\ \/grant\ Everyone\:F\ \/T\ \/C\ \/Q* OR *bcdedit\ \/set\ \{default\}\ recoveryenabled\ no* OR *wbadmin\ delete\ catalog\ \-quiet* OR *@Please_Read_Me@.txt*))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()